In [1]:
!pip install pykan


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

from utils import *

node_coords = pd.read_csv("data/sioux/SiouxFalls_node.tntp", sep='\t')

sioux = create_network_df(network_name="SiouxFalls")
T_0, C = prepare_network_data(sioux)

directory = "/home/podozerovapo/traffic_assignment/data/sioux/uncongested"

inputs = []
outputs = []
metadata = []

for filename in sorted(os.listdir(directory)):
    if filename.endswith(".pkl"):
        filepath = os.path.join(directory, filename)
        
        with open(filepath, 'rb') as f:
            data_pair = pickle.load(f)
            
            inputs.append(data_pair['input'])
            outputs.append(data_pair['output'])
            metadata.append(data_pair.get('metadata', None))

input_matrices = np.array(inputs)  # [num_samples, num_nodes, num_nodes]
output_matrices = np.array(outputs)  # [num_samples, num_nodes, num_nodes]


class FeatureEmbedding(nn.Module):
    def __init__(self, input_size, embedding_size=32):
        super(FeatureEmbedding, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, embedding_size),
            nn.ReLU()
        )

    def forward(self, x):
        return self.network(x)
    
node_coords_arr = np.array(node_coords[['X', 'Y']])
node_coords_arr.shape

np.expand_dims(node_coords_arr, 0)

ModuleNotFoundError: No module named 'pandas'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

ModuleNotFoundError: No module named 'torch'

In [ ]:

class TrafficGCNN(nn.Module):
    def __init__(self, num_nodes, num_edges, device='cpu'):
        """
        GCNN for Traffic Assignment Problem
        
        Args:
            num_nodes (int): Number of nodes in the network
            num_edges (int): Number of edges in the network
            device (str): Computation device ('cpu' or 'cuda')
        """
        super(TrafficGCNN, self).__init__()
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.device = device
        
        # Layer 1: Graph Convolution Layer
        self.theta = nn.Parameter(torch.randn(num_nodes, num_nodes))
        
        # Layer 2: Flow Distribution Layer
        self.W_q = nn.Parameter(torch.randn(num_nodes, num_edges))
        
        # Output Layer: Flow Aggregation Layer
        self.W_F = nn.Parameter(torch.randn(num_nodes))
        
    def forward(self, X, A_w, D_w_bar):
        """
        Forward pass of the GCNN
        
        Args:
            X (torch.Tensor): OD demand matrix [batch_size, num_nodes, num_nodes]
            A_w (torch.Tensor): Weighted adjacency matrix [num_nodes, num_nodes]
            D_w_bar (torch.Tensor): Weighted degree matrix with self-loops [num_nodes, num_nodes]
            
        Returns:
            torch.Tensor: Predicted link flows [batch_size, num_edges]
        """
        # Layer 1: Graph Convolution with diffusion process
        # Compute normalized adjacency with self-loops
        A_w_bar = A_w + torch.eye(self.num_nodes).to(self.device)
        transition_matrix = torch.inverse(D_w_bar) @ A_w_bar
        
        # Diffusion convolution operation (Equation 11)
        H1 = torch.tanh(self.theta @ transition_matrix @ X)
        
        # Layer 2: Flow distribution to links
        H2 = torch.tanh(H1 @ self.W_q)
        
        # Output Layer: Flow aggregation
        F = (H2.transpose(1, 2) @ self.W_F)
        
        return F
    
    def laplacian_forward(self, X, A_w, D_w_bar):
        """
        Alternative forward pass using Laplacian matrix (Equation 14)
        """
        # Compute normalized Laplacian
        A_w_bar = A_w + torch.eye(self.num_nodes).to(self.device)
        L_w_bar = D_w_bar - A_w_bar
        norm_lap = torch.inverse(D_w_bar) @ L_w_bar
        
        # Laplacian-based convolution
        H1 = torch.tanh(self.theta @ (torch.eye(self.num_nodes).to(self.device) - norm_lap) @ X)
        
        # Rest of the network remains the same
        H2 = torch.tanh(H1 @ self.W_q)
        F = (H2.transpose(1, 2) @ self.W_F)
        
        return F


class TrafficAssignmentModel:
    def __init__(self, num_nodes, num_edges, device='cpu'):
        self.gcnn = TrafficGCNN(num_nodes, num_edges, device)
        self.device = device
        self.gcnn.to(device)
        
    def train(self, dataset, epochs=100, lr=0.001, batch_size=32):
        """
        Train the GCNN model
        
        Args:
            dataset: TrafficDataset object containing:
                - OD matrices
                - Capacity matrices (C)
                - Free-flow time matrices (T0)
                - Ground truth link flows
            epochs (int): Number of training epochs
            lr (float): Learning rate
            batch_size (int): Batch size for training
        """
        optimizer = torch.optim.Adam(self.gcnn.parameters(), lr=lr)
        criterion = nn.MSELoss()
        
        for epoch in range(epochs):
            total_loss = 0
            for batch in dataset.get_batches(batch_size):
                # Prepare batch data
                X_batch = batch['od_matrix'].to(self.device)
                C_batch = batch['capacity'].to(self.device)
                T0_batch = batch['free_flow_time'].to(self.device)
                y_batch = batch['link_flows'].to(self.device)
                
                # Create weighted adjacency matrix (A_w)
                # Using inverse of free-flow time as weights
                A_w = self._create_weighted_adjacency(T0_batch)
                
                # Create degree matrix (D_w_bar)
                D_w_bar = self._create_degree_matrix(A_w)
                
                # Forward pass
                pred_flows = self.gcnn(X_batch, A_w, D_w_bar)
                
                # Compute loss
                loss = criterion(pred_flows, y_batch)
                
                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                total_loss += loss.item()
            
            print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataset):.4f}")
    
    def _create_weighted_adjacency(self, T0):
        """
        Create weighted adjacency matrix from free-flow time
        Weights are inverse of free-flow time (Equation 1 in paper)
        """
        # Add small constant to avoid division by zero
        weights = 1 / (T0 + 1e-6)
        # Zero out diagonal (no self-loops here, they'll be added later)
        mask = torch.eye(T0.size(1)).to(self.device)
        return weights * (1 - mask)
    
    def _create_degree_matrix(self, A_w):
        """
        Create degree matrix D_w_bar from weighted adjacency matrix
        with self-loops (Equation 7)
        """
        A_w_bar = A_w + torch.eye(A_w.size(1)).to(self.device)
        return torch.diag(A_w_bar.sum(dim=1))
    
    def predict(self, X, C, T0):
        """
        Predict link flows for given OD matrix, capacity and free-flow time
        """
        self.gcnn.eval()
        with torch.no_grad():
            X = X.to(self.device)
            C = C.to(self.device)
            T0 = T0.to(self.device)
            
            A_w = self._create_weighted_adjacency(T0)
            D_w_bar = self._create_degree_matrix(A_w)
            
            return self.gcnn(X, A_w, D_w_bar)


class TrafficDataset(torch.utils.data.Dataset):
    """
    Dataset class for traffic assignment problem
    """
    def __init__(self, od_matrices, capacities, free_flow_times, link_flows):
        """
        Args:
            od_matrices: List/array of OD demand matrices [num_samples, num_nodes, num_nodes]
            capacities: List/array of capacity matrices [num_samples, num_edges]
            free_flow_times: List/array of free-flow time matrices [num_samples, num_nodes, num_nodes]
            link_flows: List/array of ground truth link flows [num_samples, num_edges]
        """
        self.od_matrices = torch.tensor(od_matrices, dtype=torch.float32)
        self.capacities = torch.tensor(capacities, dtype=torch.float32)
        self.free_flow_times = torch.tensor(free_flow_times, dtype=torch.float32)
        self.link_flows = torch.tensor(link_flows, dtype=torch.float32)
        
    def __len__(self):
        return len(self.od_matrices)
    
    def __getitem__(self, idx):
        return {
            'od_matrix': self.od_matrices[idx],
            'capacity': self.capacities[idx],
            'free_flow_time': self.free_flow_times[idx],
            'link_flows': self.link_flows[idx]
        }
    
    def get_batches(self, batch_size):
        dataloader = torch.utils.data.DataLoader(
            self, batch_size=batch_size, shuffle=True
        )
        return dataloader